In [1]:
import sqlite3

import textdistance
from Levenshtein import jaro_winkler

# Connect to the SQLite database (or create it)
conn = sqlite3.connect('university.db',timeout=10)
cursor = conn.cursor()
# Drop tables if they exist (to reset the database for fresh creation)
cursor.execute("DROP TABLE IF EXISTS department")
cursor.execute("DROP TABLE IF EXISTS course")
cursor.execute("DROP TABLE IF EXISTS student")
cursor.execute("DROP TABLE IF EXISTS instructor")
cursor.execute("DROP TABLE IF EXISTS section")
cursor.execute("DROP TABLE IF EXISTS teaches")
cursor.execute("DROP TABLE IF EXISTS classroom")
cursor.execute("DROP TABLE IF EXISTS takes")
cursor.execute("DROP TABLE IF EXISTS advisor")
cursor.execute("DROP TABLE IF EXISTS prereq")

cursor.execute('''
    CREATE TABLE department (
        dept_name TEXT PRIMARY KEY,
        building TEXT,
        budget REAL
    )
''')

cursor.execute('''
    CREATE TABLE course (
        course_id TEXT PRIMARY KEY,
        title TEXT,
        dept_name TEXT,
        credits INTEGER,
        FOREIGN KEY (dept_name) REFERENCES department(dept_name)
    )
''')

cursor.execute('''
    CREATE TABLE student (
        ID TEXT PRIMARY KEY,
        name TEXT,
        dept_name TEXT,
        tot_cred INTEGER,
        FOREIGN KEY (dept_name) REFERENCES department(dept_name)
    )
''')

cursor.execute('''
    CREATE TABLE instructor (
        ID TEXT PRIMARY KEY,
        name TEXT,
        dept_name TEXT,
        salary REAL,
        FOREIGN KEY (dept_name) REFERENCES department(dept_name)
    )
''')

cursor.execute('''
    CREATE TABLE section (
        course_id TEXT,
        sec_id TEXT,
        semester TEXT CHECK (semester IN ('Q1', 'Q2', 'Q3', 'Q4')), -- Semester constraint
        year INTEGER CHECK (year > 2000), -- Year constraint
        building TEXT,
        room_no TEXT,
        time_slot_id TEXT CHECK (time_slot_id IN ('A', 'B', 'C', 'D')), -- Time slot constraint
        PRIMARY KEY (course_id, sec_id, semester, year),
        FOREIGN KEY (course_id) REFERENCES course(course_id)
    )
''')

cursor.execute('''
    CREATE TABLE teaches (
        ID TEXT,
        course_id TEXT,
        sec_id TEXT,
        semester TEXT CHECK (semester IN ('Q1', 'Q2', 'Q3', 'Q4')), -- Semester constraint
        year INTEGER CHECK (year > 2000), -- Year constraint
        PRIMARY KEY (ID, course_id, sec_id, semester, year),
        FOREIGN KEY (ID) REFERENCES instructor(ID),
        FOREIGN KEY (course_id, sec_id, semester, year) REFERENCES section(course_id, sec_id, semester, year)
    )
''')

cursor.execute('''
    CREATE TABLE classroom (
        building TEXT,
        room_no TEXT,
        capacity INTEGER,
        PRIMARY KEY (building, room_no)
    )
''')

cursor.execute('''
    CREATE TABLE takes (
        ID TEXT,
        course_id TEXT,
        sec_id TEXT,
        semester TEXT CHECK (semester IN ('Q1', 'Q2', 'Q3', 'Q4')), -- Semester constraint
        year INTEGER CHECK (year > 2000), -- Year constraint
        grade TEXT,
        PRIMARY KEY (ID, course_id, sec_id, semester, year),
        FOREIGN KEY (ID) REFERENCES student(ID),
        FOREIGN KEY (course_id, sec_id, semester, year) REFERENCES section(course_id, sec_id, semester, year)
    )
''')

cursor.execute('''
    CREATE TABLE advisor (
        s_id TEXT,
        i_id TEXT,
        PRIMARY KEY (s_id),
        FOREIGN KEY (s_id) REFERENCES student(ID),
        FOREIGN KEY (i_id) REFERENCES instructor(ID)
    )
''')

cursor.execute('''
    CREATE TABLE prereq (
        course_id TEXT,
        prereq_id TEXT,
        PRIMARY KEY (course_id, prereq_id),
        FOREIGN KEY (course_id) REFERENCES course(course_id),
        FOREIGN KEY (prereq_id) REFERENCES course(course_id)
    )
''')

# Commit the changes and close the connection
conn.commit()
conn.close()
print("Database created successfully!")


Database created successfully!


In [2]:
#Q2
conn = sqlite3.connect('university.db')
cursor = conn.cursor()
cursor.execute(""" INSERT INTO student (ID,name,dept_name,tot_cred) VALUES ("1","Jason","Computer Science",3);""")
cursor.execute("""INSERT INTO student (ID,name,dept_name,tot_cred) VALUES ("2","Jad","Biology",3);""")
cursor.execute("""INSERT INTO student (ID,name,dept_name,tot_cred) VALUES ("3","Nizar","Architecture",3);""")
cursor.execute("""  INSERT INTO student (ID,name,dept_name,tot_cred) VALUES ("4","Nason","Computer Engineering",3); """)
conn.commit()
conn.close()

In [6]:
import pandas as pd
conn = sqlite3.connect('university.db')
cursor = conn.cursor()
df= pd.read_sql_query("SELECT * FROM student", conn)
df

,ID,name,dept_name,tot_cred
0,1,Jason,Computer Science,3
1,2,Jad,Biology,3
2,3,Nizar,Architecture,3
3,4,Nason,Computer Engineering,3


In [10]:
import pandas as pd
import sqlite3
from itertools import combinations

# Load data from the SQLite database into a DataFrame
conn = sqlite3.connect('university.db')
table_name = 'student'
df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
conn.close()

# Ensure no leading/trailing spaces in column names
df.columns = df.columns.str.strip()

# Check if X -> Y (X functionally determines Y)
def check_fd(df, X, Y):
    try:
        # Group by the determinant (X) and check if Y has multiple unique values
        grouped = df.groupby(list(X))[Y].nunique()
        return (grouped <= 1).all()
    except KeyError as e:
        print(f"KeyError: {e} - One of the columns in {X} or {Y} is missing.")
        return False
    except Exception as e:
        print(f"Error: {e} - Something went wrong with grouping or checking functional dependency.")
        return False

# Function to find all functional dependencies
def find_functional_dependencies(df):
    fds = []
    columns = df.columns
    
    # Iterate over all possible combinations of column sets X -> Y
    for k in range(1, len(columns)):
        for X in combinations(columns, k):
            for Y in columns:
                if Y not in X:
                    if check_fd(df, X, Y):
                        fds.append(f"{X} -> {Y}")
    
    return fds

# Find functional dependencies
fds = find_functional_dependencies(df)

# Print the discovered FDs
for fd in fds:
    print(fd)


('ID',) -> name
('ID',) -> dept_name
('ID',) -> tot_cred
('name',) -> ID
('name',) -> dept_name
('name',) -> tot_cred
('dept_name',) -> ID
('dept_name',) -> name
('dept_name',) -> tot_cred
('ID', 'name') -> dept_name
('ID', 'name') -> tot_cred
('ID', 'dept_name') -> name
('ID', 'dept_name') -> tot_cred
('ID', 'tot_cred') -> name
('ID', 'tot_cred') -> dept_name
('name', 'dept_name') -> ID
('name', 'dept_name') -> tot_cred
('name', 'tot_cred') -> ID
('name', 'tot_cred') -> dept_name
('dept_name', 'tot_cred') -> ID
('dept_name', 'tot_cred') -> name
('ID', 'name', 'dept_name') -> tot_cred
('ID', 'name', 'tot_cred') -> dept_name
('ID', 'dept_name', 'tot_cred') -> name
('name', 'dept_name', 'tot_cred') -> ID


In [2]:
!pip install levenshtein

In [8]:
import Levenshtein as ls
str1="Computation"
str2 = "Completion"
#Q5-a: 
print("Q5-a:")
print(ls.distance(str1,str2,weights=(1,1,1)))
#Q5-b:
print("Q5-b:")
print(ls.distance(str1,str2,weights=(1,1,2)))
#Q5-c:
print("Q5-c:")
print(ls.distance(str1,str2,weights=(1,1,3)))
#What do you expect if the cost of the update operation is > 3 while the cost of insert and delete remains 1? Explain your answer.
#When the cost of substitution exceeds 3, I expect the total edit distance to remain constant at 5. Since the substitution operation becomes too expensive, the algorithm will try to avoid using it as much as possible. Instead, it will rely more on insertions and deletions, both of which still cost 1. As a result, the algorithm can minimize the total cost by favoring these cheaper operations 
# We can demonstrate this by increasing the cost of either deletion or insertion. When the cost of these operations exceeds 1, the edit distance becomes greater than 5, as the algorithm is forced to use more costly operations.

Q5-a:
3
Q5-b:
5
Q5-c:
5
5
10


In [59]:
!pip install biopython

In [63]:
#This python library was suggested to me by ChatGPT. I asked what libraries can I use to calculate Gap Distance in py?
from Bio import pairwise2
from Bio.pairwise2 import format_alignment 
str1 = "Advances in Instrumentation and Control"
str2 = "Adv. Instrum. Control"
alignments = pairwise2.align.globalxs(str1, str2,open =-1, extend =-0.1)
for alignment in alignments:
    print(format_alignment(*alignment))

Advances in Instrumentation and Control
|||       .||||||||           .||||||||
Adv-------. Instrum-----------. Control
  Score=15.4

Advances in Instrumentation and Control
|||.       ||||||||           .||||||||
Adv.------- Instrum-----------. Control
  Score=15.4

Advances in Instrumentation and Control
|||       .||||||||.           ||||||||
Adv-------. Instrum.----------- Control
  Score=15.4

Advances in Instrumentation and Control
|||.       ||||||||.           ||||||||
Adv.------- Instrum.----------- Control
  Score=15.4



In [64]:
!pip install textdistance

  Obtaining dependency information for textdistance from https://files.pythonhosted.org/packages/c6/c2/c62601c858010b0513a6434b9be19bd740533a6e861eddfd30b7258d92a0/textdistance-4.6.3-py3-none-any.whl.metadata


In [69]:
import textdistance as td
A = {0,1,2,5,6}
B = {0,2,3,5,7,9}
C = {2,3,5,6}
ab = td.jaccard.distance(A,B)
ac = td.jaccard.distance(A,C)
bc = td.jaccard.distance(B,C)
print(ab,ac,bc)

0.625 0.5 0.5714285714285714


In [74]:
import textdistance

# Define the sets as multisets (bags)
A = [1, 1, 2, 2, 5]
B = [1, 2, 2, 2, 5, 5]
C = [1, 2, 3, 4, 5]

# Function to compute Jaccard similarity using textdistance
def jaccard_bag_similarity(set1, set2):
    return textdistance.Jaccard(qval=1).similarity(set1, set2)

# Compute pairwise Jaccard bag similarities
similarity_A_B = jaccard_bag_similarity(A, B)
similarity_A_C = jaccard_bag_similarity(A, C)
similarity_B_C = jaccard_bag_similarity(B, C)

# Output the results
print(f"Jaccard bag similarity (A, B): {similarity_A_B}")
print(f"Jaccard bag similarity (A, C): {similarity_A_C}")
print(f"Jaccard bag similarity (B, C): {similarity_B_C}")


Jaccard bag similarity (A, B): 0.5714285714285714
Jaccard bag similarity (A, C): 0.42857142857142855
Jaccard bag similarity (B, C): 0.375


In [81]:
str1 = "arnab"
str2 = "ubran"

jaro = td.jaro.similarity(str1, str2)
print(jaro)
jaro_winkler = td.jaro_winkler.similarity(str1, str2)
print(jaro_winkler)
#What do you think is the reason for this result?

0.6
0.6


In [84]:
def word_based_shingles(text, k=5):
    words = text.split()
    shingles = set()
    # Generate all k-word shingles
    for i in range(len(words) - k + 1):
        shingles.add(' '.join(words[i:i+k]))
    return shingles
sentence = "Many problems can be expressed as finding similar sets"
print(f"{word_based_shingles(sentence)}, Therefore the cardinality of the 5-shingle set is {len(word_based_shingles(sentence))}")

{'Many problems can be expressed', 'expressed as finding similar sets', 'be expressed as finding similar', 'can be expressed as finding', 'problems can be expressed as'}, Therefore the cardinality of the 5-shingle set is 5
